# Categorizing urban density

last modified: February, 2016

author: [Raj Singh](https://developer.ibm.com/clouddataservices/author/rrsingh/)

original: https://github.com/ibm-cds-labs/open-data/blob/master/samples/urbanity.ipynb


## Overview

Exploration of an academic study of urban structure and density described in the June 2014 article, ["From Jurisdictional to Functional Analysis of Urban Cores & Suburbs"](http://www.newgeography.com/content/004349-from-jurisdictional-functional-analysis-urban-cores-suburbs) in [new geography](http://www.newgeography.com/). 

## Categories

- Urban (pre-auto urban core): density > 2,900 sq. km
- Auto suburban, early: median house built 1946 to 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto suburban, later: median house built after 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto exurban: all others


In [1]:
import pandas as pd, numpy as np

## Collect U.S. Census data

Census data from the 2013 US Census American Community Survey (ACS), 5-year estimates.

Created from the "zip code tabulation area" (ZCTA) [TIGER/Line® with Selected Demographic and Economic Data product in Geodatabase format](http://www.census.gov/geo/maps-data/data/tiger-data.html). This particular version of the ACS is used for the folowing reasons:

1. 5-year estimates are the most accurate data outside of the decennial census [as explained here](http://www.census.gov/programs-surveys/acs/guidance/estimates.html).
1. 2013 is the most recent data set with 5-year estimates
1. TIGER/Line® gives you the geographic boundaries of the zip codes so you can perform spatial analyses
1. This data set is smaller than the full Census, but still has the important income, education, race, age and occupation demographics we want to use.

If you want to do this yourself, [this article](https://developer.ibm.com/clouddataservices/2015/09/08/census-open-data-on-ibm-cloud/) explains how to get a CSV out of that format.


### Get zip code areas from Census

In [2]:
geo_df = pd.read_csv('http://opendata.mybluemix.net/uscensus/areas.csv.gz', compression='gzip', 
                    usecols=['GEOID10','ALAND10'], dtype={"GEOID10": np.str})
geo_df['GEOID'] = "86000US" + geo_df['GEOID10']
geo_df = geo_df.set_index('GEOID')
geo_df.head()

,GEOID10,ALAND10
GEOID,,
86000US43451,43451,63411475
86000US43452,43452,121783680
86000US43456,43456,9389360
86000US43457,43457,48035540
86000US43458,43458,2573816


### Get population from Census

In [3]:
pop_df = pd.read_csv('http://opendata.mybluemix.net/uscensus/x01_age_sex.csv.gz', compression='gzip', 
                     usecols=['GEOID','B01001e1'], dtype={"GEOID": np.str})
pop_df = pop_df.set_index('GEOID')
pop_df.head()

,B01001e1
GEOID,
86000US01001,17245
86000US01002,29266
86000US01003,11032
86000US01005,5356
86000US01007,14673


### Get housing age from Census

In [4]:
housing_df = pd.read_csv('http://opendata.mybluemix.net/uscensus/x25_housing.csv.gz', compression='gzip', 
                     usecols=['GEOID','B25035e1'], dtype={"GEOID": np.str})
housing_df = housing_df.set_index('GEOID')
housing_df.sample(5)

,B25035e1
GEOID,
86000US61114,1981
86000US41861,1991
86000US60451,1991
86000US50246,1939
86000US29409,1939


### Join Census data into one DataFrame with nice names

In [5]:
urban_df = geo_df.join(pop_df)
urban_df = urban_df.join(housing_df)

In [6]:
urban_df.columns = ['ZIP','AREAMSQ','Population','MEDYRBUILT']
urban_df.sample(5)

,ZIP,AREAMSQ,Population,MEDYRBUILT
GEOID,,,,
86000US17946,17946,4950554,215,1939
86000US85006,85006,10110421,25097,1951
86000US87562,87562,150116088,435,1993
86000US36693,36693,32638329,17338,1977
86000US45884,45884,434353,159,1939


## Density calculation
persons per square km = persons / (area in square meters / 1,000,000)
persons per hectare = persons / (area in square meters / 10,000)

### Compute population density as persons per square kilometer

In [7]:
urban_df['POPPERKMSQ'] = urban_df['Population'] / (urban_df['AREAMSQ']/1000000)
urban_df.sample(4)

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
GEOID,,,,,
86000US74114,74114,12557398,15795,1953,1257.824272
86000US58734,58734,487335570,271,1967,0.556085
86000US13673,13673,90395985,2229,1978,24.658175
86000US57460,57460,310706112,364,1939,1.171525


### Group population density into 4 categories

In [8]:
urban_df['CAT'] = 'EXURBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] >= 2900)] = 'URBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] < 1980) & (urban_df['MEDYRBUILT'] >= 1946)] = 'SUBURBANEARLY'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] >= 1980)] = 'SUBURBANLATE'
urban_df.describe()

/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
count,3.298900e+04,32989.000000,32045.000000,32989.000000
mean,2.250019e+08,9443.177453,1971.068529,487.703913
std,6.575933e+08,13858.010530,15.606758,1912.093435
min,5.094000e+03,0.000000,1939.000000,0.000000
25%,2.351832e+07,719.000000,1961.000000,7.754449
50%,9.322068e+07,2781.000000,1974.000000,30.194351
75%,2.304373e+08,12830.000000,1982.000000,249.247358
max,3.478591e+10,114734.000000,2011.000000,71226.281507


In [9]:
# look at a few records to do a quick sanity check
urban_df.sample(30)

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,,
86000US41839,41839,76295717,1319,1983,17.287995,EXURBAN
86000US23421,23421,83707372,5078,1982,60.663713,EXURBAN
86000US43210,43210,4003194,10860,1974,2712.833802,SUBURBANEARLY
86000US58316,58316,305498371,6726,1978,22.016484,EXURBAN
86000US40229,40229,51148335,36741,1979,718.322503,SUBURBANEARLY
86000US76357,76357,171874142,626,1955,3.642200,EXURBAN
86000US03441,03441,10142581,584,1939,57.579032,EXURBAN
86000US84078,84078,2402057201,27369,1984,11.393983,EXURBAN
86000US56111,56111,129874170,467,1952,3.595788,EXURBAN


## Mapping Urbanity

In [11]:
#!pip install --user cartodb
# Set up cartodb module for use
from cartodb import CartoDBAPIKey, CartoDBException
# Learn how to use the CartoDB SQL API here:
# http://docs.cartodb.com/cartodb-platform/sql-api/making-calls/
API_KEY = 'dfd11d7ad9ad3f213a6336b5e878865422f624f1'
CARTODB_ACCOUNT = 'ibm'
cl = CartoDBAPIKey(API_KEY, CARTODB_ACCOUNT)

### Update data table in CartoDB

In [12]:
# get all non-exurban zips
notex_df = urban_df[ urban_df['CAT'] != 'EXURBAN' ]
notex_df.head()

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,,
86000US43452,43452,121783680,13659,1974,112.157885,SUBURBANEARLY
86000US43458,43458,2573816,312,1960,121.220787,SUBURBANEARLY
86000US43460,43460,7158543,6334,1959,884.816924,SUBURBANEARLY
86000US43465,43465,28559486,5267,1974,184.422087,SUBURBANEARLY
86000US43468,43468,1861424,403,1957,216.500915,SUBURBANEARLY


In [13]:
# generate PostgreSQL INSERT statements for CartoDB
insertsql = ''
for index, row in notex_df.iterrows(): 
    insertsql += "INSERT INTO urbanity(zipcode,category) VALUES ('" + index + "', '" + row['CAT'] + "');"

In [14]:
# send SQL INSERT statements to CartoDB
try:
    print(cl.sql('delete from urbanity'))
    print(cl.sql(insertsql))
except CartoDBException as e:
    print("some error ocurred", e)

{u'fields': {}, u'rows': [], u'total_rows': 10002, u'time': 0.123}
{u'fields': {}, u'rows': [], u'total_rows': 1, u'time': 73.506}


### Urbanity Map!

In [15]:
%%javascript
element.append("<link rel='stylesheet' href='http://libs.cartocdn.com/cartodb.js/v3/3.15/themes/css/cartodb.css' />")
element.append("<div id='map' style='height:500px;width:900px;padding:0;margin:0'></div>");

require.config({
  paths: {
      cartodblib: 'http://libs.cartocdn.com/cartodb.js/v3/3.15/cartodb'
  }
});

var main = function() {
  cartodb.createVis('map', 'https://ibmanalytics.cartodb.com/u/ibm/api/v2/viz/3a4e9e24-b3f6-11e5-a4c7-0e31c9be1b51/viz.json', {
      shareable: true,title: true,description: true,search: true,tiles_loader: true,
      center_lat: 40, center_lon: -100, zoom: 3
  })
  .done(function(vis, layers) {
    // layer 0 is the base layer, layer 1 is cartodb layer
    // setInteraction is disabled by default
    layers[1].setInteraction(true);
    layers[1].on('featureOver', function(e, latlng, pos, data) {
      cartodb.log.log(e, latlng, pos, data);
    });
    
    // var map = vis.getNativeMap(); // get the native map to work with it
    
  })
  .error(function(err) {
    console.log(err);
  });
}

require(['cartodblib'], main);

<IPython.core.display.Javascript object>